# Goal
- build a file authentication system that lets browsers authenticate and play video chunks without having to wait for the entire file.


# Notes

### `seek(offset, from_what)`
- sets the file's current position at the offset
- from_what <- optional defaults to zero
- 0 - absolute position 
- 1 - relative to current position 
- 2 - seek relative to file's end  (`os.SEEK_END`)

### `tell()`
- return's current positon of the file read/write pointer within the file


### `tester.txt`
- `buttersize = 4`
- `size = 94 bytes.`
```
the quick brown fox jumps over the lazy dog. YES, THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG.
```

In [1]:
from hashlib import sha256

x = sha256(b"abc123").hexdigest()
print(x)

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"abc123")
z = digest.finalize()
y = b'l\xa1=R\xcap\xc8\x83\xe0\xf0\xbb\x10\x1eBZ\x89\xe8bM\xe5\x1d\xb2\xd29%\x93\xafj\x84\x11\x80\x90'

print(z == y)
print(z.hex() == x)

6ca13d52ca70c883e0f0bb101e425a89e8624de51db2d2392593af6a84118090
6ca13d52ca70c883e0f0bb101e425a89e8624de51db2d2392593af6a84118090
6ca13d52ca70c883e0f0bb101e425a89e8624de51db2d2392593af6a84118090
6ca13d52ca70c883e0f0bb101e425a89e8624de51db2d2392593af6a84118090
6ca13d52ca70c883e0f0bb101e425a89e8624de51db2d2392593af6a84118090
True
True
True
True
True
True
True
True
True
True


In [2]:
import os

def read_block_reverse1(filepath, buffersize=1024):

    with open(filepath, 'rb') as f:

        f.seek(0, os.SEEK_END)
        filesize = f.tell()
        firstchunk = filesize % buffersize
        position = filesize - firstchunk
        if firstchunk != 0: 
            f.seek(position)
            yield f.read(firstchunk)
        
        while position > 0:
            position -= buffersize
            f.seek(position)
            yield f.read(buffersize)

            
def read_block_reverse2(filepath, buffersize=1024):

    with open(filepath, 'rb') as f:
        
        f.seek(0, os.SEEK_END)
        filesize = f.tell()
        firstchunk = filesize % buffersize

        if firstchunk != 0: 
            f.seek(filesize - firstchunk)
            yield f.read(firstchunk)
        
        f.seek(-firstchunk-buffersize, os.SEEK_END)
        
        while True:
            yield f.read(buffersize)
            if f.tell() <= buffersize:
                break
            f.seek(-2*buffersize, 1)
    
def test1():
    gen = read_block_reverse1(filepath='./data/6.2.birthday.mp4_download', buffersize=1024)
    for i in gen:
        print("", end="")
        
def test2():
    gen2 = read_block_reverse2(filepath='./data/6.2.birthday.mp4_download', buffersize=1024)
    for i in gen2:
        print("", end="")


if __name__ == '__main__':
    import timeit
    print("test1", timeit.timeit("test1()", setup="from __main__ import test1", number=100))
    print("test2", timeit.timeit("test2()", setup="from __main__ import test2", number=100))

test1 311.64912715699757
test1 311.64912715699757
test1 311.64912715699757
test1 311.64912715699757
test1 311.64912715699757
test2 256.4095842000097
test2 256.4095842000097
test2 256.4095842000097
test2 256.4095842000097
test2 256.4095842000097
